In [2]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [3]:
import time
import sys
sys.path.insert(0,'../..')
from keras import losses, metrics
from tensorflow.keras import optimizers
import gp2
from gp2 import Runner

In [4]:
help(gp2.KVNet2D)

Help on class KVNet2D in module gp2.gp2.classifiers.k_vnet2d:

class KVNet2D(gp2.gp2.classifiers.base_keras_segmentation_classifier.BaseKerasSegmentationClassifier)
 |  KVNet2D(input_size=(512, 512, 1), filter_num=None, n_labels=1, res_num_ini=1, res_num_max=2, activation='ReLU', output_activation='Sigmoid', batch_norm=True, pool=False, unpool=False, name='vnet', optimizer=None, loss=None, metric=None, verbose=False, workingdir='/tmp')
 |  
 |  KVNet2D for binary segmentation
 |  
 |  Method resolution order:
 |      KVNet2D
 |      gp2.gp2.classifiers.base_keras_segmentation_classifier.BaseKerasSegmentationClassifier
 |      gp2.gp2.classifiers.classifier.Classifier
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, input_size=(512, 512, 1), filter_num=None, n_labels=1, res_num_ini=1, res_num_max=2, activation='ReLU', output_activation='Sigmoid', batch_norm=True, pool=False, unpool=False, name='vnet', optimizer=None, loss=None, metric=None, verbose=False, w

In [5]:
R = Runner(verbose=True,
           classifier='kvnet2d',
           discriminator='cnn',
           filter_num=[16, 32, 64, 128],
           res_num_ini=1, 
           res_num_max=2, 
           activation='ReLU', 
           output_activation='Sigmoid', 
           batch_norm=True, 
           pool=False, 
           unpool=False,
           optimizer=None, 
           loss=None, 
           metric=None,
          )

2023-04-23 14:51:33.709799: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 14:51:34.035790: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:07:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-04-23 14:51:34.038408: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:0f:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-04-23 14:51:34.040781: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 

*** GP2  KVNet2D ***
Working directory: /tmp/tmpqyg2j1d9GP2
Verbose mode active!
{'verbose': True, 'workingdir': '/tmp/tmpqyg2j1d9GP2', 'input_size': (512, 512, 1), 'filter_num': [16, 32, 64, 128], 'n_labels': 1, 'res_num_ini': 1, 'res_num_max': 2, 'activation': 'ReLU', 'output_activation': 'Sigmoid', 'batch_norm': True, 'pool': False, 'unpool': False, 'name': 'vnet', 'optimizer': <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7f34b3184190>, 'loss': <function binary_crossentropy at 0x7f35151678b0>, 'metric': [<function Util.dice_coef at 0x7f3514177280>], 'model': <tensorflow.python.keras.engine.functional.Functional object at 0x7f34b00ad6a0>}
Model summary:
Model: "vnet_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
____

In [6]:
# load our larger toy dataset (10k images and masks)
images = np.load('/hpcstor6/scratch01/r/ryan.zurrin001/GP2TOYEXAMPLE_LARGE/images.npy')
masks = np.load('/hpcstor6/scratch01/r/ryan.zurrin001/GP2TOYEXAMPLE_LARGE/masks.npy')

In [7]:
images = images[:5000]
masks = masks[:5000]

In [8]:
images.shape

(5000, 512, 512, 1)

In [9]:
masks.shape

(5000, 512, 512, 1)

In [12]:
weights = {
    'A': 0.5,
    'A_train': 0.1,
    'A_val': 0.3,
    'A_test': 0.6,
    'B': 0.3,
    'B_train': 0.7,
    'B_val': 0.1,
    'B_test': 0.2,
    'Z': 0.2
}

In [ ]:
R.run(images=images, masks=masks, weights=weights, runs=3)

Weights OK!
******
Loop 0


2023-04-23 14:52:18.521141: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-04-23 14:52:18.538994: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2245755000 Hz


Epoch 1/100


2023-04-23 14:52:20.831299: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-04-23 14:52:21.579937: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2023-04-23 14:52:22.336692: W tensorflow/stream_executor/gpu/asm_compiler.cc:191] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.0
2023-04-23 14:52:22.336716: W tensorflow/stream_executor/gpu/asm_compiler.cc:194] Used ptxas at ptxas
2023-04-23 14:52:22.336773: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-04-23 14:52:22.352462: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-04-23 14:52:23.196414: I tensorflow/stream_exec

4/4 [==============================] - 24s 4s/step - loss: 0.8487 - dice_coef: 0.1516 - val_loss: 1.6684 - val_dice_coef: 0.1328


...

Epoch 1/100
6/6 [==============================] - 7s 1s/step - loss: 0.8487 - dice_coef: 0.1530 - val_loss: 0.8643 - val_dice_coef: 0.1457


/home/ryan.zurrin001/miniconda3/envs/O/lib/python3.9/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/100
6/6 [==============================] - 4s 675ms/step - loss: 0.8487 - dice_coef: 0.1524 - val_loss: 0.8529 - val_dice_coef: 0.1472


/home/ryan.zurrin001/miniconda3/envs/O/lib/python3.9/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 3/100
6/6 [==============================] - 4s 707ms/step - loss: 0.8486 - dice_coef: 0.1523 - val_loss: 0.8444 - val_dice_coef: 0.1486


In [ ]:
R.classifier_scores

In [ ]:
R.discriminator_scores

In [ ]:
R.plot()